In [ ]:
# Install PyIQA for No-Reference Metrics (NIQE)
!pip install pyiqa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Part 0: Environment Setup & Drive Mounting
from google.colab import drive
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define the project root directory in your Drive
PROJECT_ROOT = "/content/drive/My Drive/ECE253_Project"

# 3. Define sub-directories for organized storage
DIRS = {
    "GT": os.path.join(PROJECT_ROOT, "1_GroundTruth"),      # Folder for Sharp images
    "BLUR": os.path.join(PROJECT_ROOT, "2_InputBlur"),      # Folder for Blurred images
    "RL_OUT": os.path.join(PROJECT_ROOT, "3_Output_RL"),    # Folder for RL results
    "NAF_OUT": os.path.join(PROJECT_ROOT, "4_Output_NAFNet"), # Folder for NAFNet results
    "NAF_FT_OUT": os.path.join(PROJECT_ROOT, "5_Output_NAFNet_FineTuned"),
    "METRICS": os.path.join(PROJECT_ROOT, "Metrics")        # Folder for CSV files
}

# 4. Create directories if they don't exist
for k, v in DIRS.items():
    os.makedirs(v, exist_ok=True)

print(f"✅ Working directory ready: {PROJECT_ROOT}")

Mounted at /content/drive
✅ Working directory ready: /content/drive/My Drive/ECE253_Project


In [ ]:
# Part 2: Traditional Method (Richardson-Lucy) - Updated with NIQE & No-GT Support
# ================================================================================
import numpy as np
import cv2
import os
import pandas as pd
from skimage import metrics, restoration
from tqdm import tqdm
import torch
import pyiqa # Required for NIQE

# 1. Initialize NIQE Metric
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(">>> Loading NIQE metric...")
try:
    niqe_metric = pyiqa.create_metric('niqe', device=device)
except Exception as e:
    print(f"⚠️ Warning: Could not load NIQE: {e}")
    niqe_metric = None

# 2. Load Kernel
kernel_path = os.path.join(PROJECT_ROOT, "motion_kernel.npy")
if not os.path.exists(kernel_path):
    raise FileNotFoundError("Kernel file not found! Please run Part 1 first.")
psf = np.load(kernel_path)

files = sorted(os.listdir(DIRS["BLUR"]))
results_rl = []

print(f">>> Starting RL Restoration on {len(files)} images...")

for f in tqdm(files, desc="Running RL"):
    # Paths
    path_blur = os.path.join(DIRS["BLUR"], f)
    path_gt = os.path.join(DIRS["GT"], f)

    # Check if GT exists
    has_gt = os.path.exists(path_gt)

    # Read Blur Image
    img_blur = cv2.cvtColor(cv2.imread(path_blur), cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

    # Restoration Loop
    img_rl = np.zeros_like(img_blur)
    for i in range(3):
        img_rl[:,:,i] = restoration.richardson_lucy(img_blur[:,:,i], psf, num_iter=20, clip=False)

    # Post-processing
    img_rl = np.clip(img_rl, 0, 1)
    img_rl_uint8 = (img_rl * 255).astype(np.uint8)

    # Save Result
    cv2.imwrite(os.path.join(DIRS["RL_OUT"], f), cv2.cvtColor(img_rl_uint8, cv2.COLOR_RGB2BGR))

    # --- Metrics Calculation ---
    row = {"Image": f, "Method": "RL", "Has_GT": has_gt}

    # 1. Calculate NIQE (for ALL images)
    if niqe_metric:
        # Convert to Tensor (NCHW, 0-1)
        out_tensor = torch.from_numpy(img_rl_uint8).permute(2,0,1).unsqueeze(0).float() / 255.0
        row["NIQE"] = niqe_metric(out_tensor.to(device)).item()
    else:
        row["NIQE"] = None

    # 2. Calculate PSNR/SSIM (Only if GT exists)
    if has_gt:
        img_gt = cv2.cvtColor(cv2.imread(path_gt), cv2.COLOR_BGR2RGB)
        # Ensure sizes match (sometimes RL output might differ slightly in float ops, but usually safe)
        row["PSNR"] = metrics.peak_signal_noise_ratio(img_gt, img_rl_uint8)
        row["SSIM"] = metrics.structural_similarity(img_gt, img_rl_uint8, channel_axis=2, win_size=3)
    else:
        row["PSNR"] = None
        row["SSIM"] = None

    results_rl.append(row)

# Save Metrics
csv_path = os.path.join(DIRS["METRICS"], "metrics_rl.csv")
pd.DataFrame(results_rl).to_csv(csv_path, index=False)
print(f"✅ Part 2 Complete: Metrics saved to {csv_path}")

>>> Loading NIQE metric...
Downloading: "https://huggingface.co/chaofengc/IQA-PyTorch-Weights/resolve/main/niqe_modelparameters.mat" to /root/.cache/torch/hub/pyiqa/niqe_modelparameters.mat



100%|██████████| 8.15k/8.15k [00:00<00:00, 17.1MB/s]


>>> Starting RL Restoration on 110 images...


Running RL: 100%|██████████| 110/110 [06:06<00:00,  3.34s/it]


✅ Part 2 Complete: Metrics saved to /content/drive/My Drive/ECE253_Project/Metrics/metrics_rl.csv
